In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from fastapi import FastAPI, HTTPException
import uvicorn
import joblib
from pydantic import BaseModel
import json
import asyncio

In [2]:
# Step 1: Create sample dataset and preprocess
data = pd.DataFrame({
    "temperature": np.random.uniform(10, 40, 100),
    "vibration": np.random.uniform(0.1, 5.0, 100),
    "humidity": np.random.uniform(30, 90, 100),
    "wind_speed": np.random.uniform(1, 25, 100)
})

data.to_csv("wind_turbine_data.csv", index=False)

# Load dataset
data = pd.read_csv("wind_turbine_data.csv")

# Selecting relevant features
features = ["temperature", "vibration", "humidity", "wind_speed"]
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[features])

# Save the scaler for future use
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [3]:
# Step 2: Prepare the dataset for LSTM model
sequence_length = 10  # Using past 10 timesteps for prediction
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i : i + sequence_length])
    y.append(data_scaled[i + sequence_length][0])  # Predicting temperature as an example
X, y = np.array(X), np.array(y)

In [4]:
# Step 3: Build LSTM Model
model = Sequential([
    Input(shape=(sequence_length, len(features))),
    LSTM(50, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss=MeanSquaredError())

In [5]:
# Step 4: Train the Model
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

# Save the trained model using the recommended format
model.save("lstm_wind_turbine.keras")

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 210ms/step - loss: 0.3079 - val_loss: 0.3149
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.2489 - val_loss: 0.2473
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1821 - val_loss: 0.1815
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1511 - val_loss: 0.1247
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1089 - val_loss: 0.1028
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1196 - val_loss: 0.1063
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1387 - val_loss: 0.1026
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.1114 - val_loss: 0.1043
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.1078 - val_loss: 0.1096
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1038 - val_loss: 0.1104
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1097 - val_loss: 0.1080
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1052 - val_loss: 0.1048


In [ ]:
# Step 5: Deploying the Model with FastAPI
app = FastAPI()

# Load the trained model and scaler
model = tf.keras.models.load_model("lstm_wind_turbine.keras", compile=False)
scaler = joblib.load("scaler.pkl")

class InputData(BaseModel):
    data: list

@app.post("/predict")
async def predict(input_data: InputData):
    try:
        input_array = np.array(input_data.data).reshape(1, sequence_length, len(features))
        prediction = model.predict(input_array)
        return {"predicted_temperature": float(prediction[0][0])}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8001)  # Change port from 8000 to 8001

INFO:     Started server process [2004]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
INFO:     127.0.0.1:63513 - "POST /predict HTTP/1.1" 200 OK
